# Datawave - Cleaning and clustering

In [1]:
# Importing relevant libraries
import pandas as pd
import numpy as np

In [2]:
# Importing dataset

df = pd.read_excel('DataWave_Cleaned_Dataset.xlsx')
df.head()

,user_id,country,age,gender,subscription_type,avg_listening_hours_per_week,total_songs_played,skip_rate,satisfaction_score,churned,Join_date_clean,New_user_id,new_avg_price
0,U1403,Nepal,23,Female,Premium,11.5,510,0.05,2.0,0,2024-03-21,U1403-Nepal-23-Female-Premium-11.5,7.99
1,U1165,Nepal,63,Male,Student,11.3,526,0.05,3.0,0,2023-08-10,U1165-Nepal-63-Male-Student-11.3,0.00
2,U1474,USA,45,Female,Premium,2.0,476,0.30,3.0,0,2022-04-05,U1474-USA-45-Female-Premium-2,7.99
3,U1478,Nigeria,16,Female,Family,12.1,529,0.20,5.0,1,2023-12-16,U1478-Nigeria-16-Female-Family-12.1,12.99
4,U1609,Ghana,29,Male,Free,6.4,477,0.20,3.0,0,2022-03-28,U1609-Ghana-29-Male-Free-6.4,0.00


In [3]:
# Finding blanks
df.isna().sum()

user_id                         10
country                          0
age                              0
gender                          79
subscription_type                0
avg_listening_hours_per_week     0
total_songs_played               0
skip_rate                        0
satisfaction_score              44
churned                          0
Join_date_clean                  0
New_user_id                      0
new_avg_price                    0
dtype: int64

# Imputing Missing Values

In [4]:
# User ID is not relevant for imputation. So we delte it, but after saving it so we can add it back later

user_id = df['user_id']
df = df.drop(['user_id', 'New_user_id'], axis = 1)

In [5]:
df.head()

,country,age,gender,subscription_type,avg_listening_hours_per_week,total_songs_played,skip_rate,satisfaction_score,churned,Join_date_clean,new_avg_price
0,Nepal,23,Female,Premium,11.5,510,0.05,2.0,0,2024-03-21,7.99
1,Nepal,63,Male,Student,11.3,526,0.05,3.0,0,2023-08-10,0.00
2,USA,45,Female,Premium,2.0,476,0.30,3.0,0,2022-04-05,7.99
3,Nigeria,16,Female,Family,12.1,529,0.20,5.0,1,2023-12-16,12.99
4,Ghana,29,Male,Free,6.4,477,0.20,3.0,0,2022-03-28,0.00


## Using KNN Imputer to impute missing values

In [7]:
# Specifying categorical columns
cat_cols = ['country', 'gender', 'subscription_type', 'satisfaction_score', 'churned', 'new_avg_price']

# Encoding the categorical values using Ordinal encoder

from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

df[cat_cols] = enc.fit_transform(df[cat_cols])



In [9]:
# Scaling the numerical columns to enable better imputation

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
num_cols = ['age', 'avg_listening_hours_per_week', 'total_songs_played', 'skip_rate']
df[num_cols] = scaler.fit_transform(df[num_cols])

In [10]:
df.head()

,country,age,gender,subscription_type,avg_listening_hours_per_week,total_songs_played,skip_rate,satisfaction_score,churned,Join_date_clean,new_avg_price
0,4.0,-1.016941,0.0,2.0,0.306573,0.486594,-1.486964,1.0,0.0,2024-03-21,2.0
1,4.0,1.609359,1.0,3.0,0.264996,1.198434,-1.486964,2.0,0.0,2023-08-10,0.0
2,8.0,0.427524,0.0,2.0,-1.668348,-1.026067,1.663383,2.0,0.0,2022-04-05,2.0
3,5.0,-1.476543,0.0,0.0,0.431305,1.331904,0.403244,4.0,1.0,2023-12-16,4.0
4,1.0,-0.622996,1.0,1.0,-0.753648,-0.981577,0.403244,2.0,0.0,2022-03-28,0.0


In [11]:
# Dates are not well recognised by KNN Imputer so we drop it after saving it so we can add it back in the future

dates = df['Join_date_clean']
df2 = df.drop('Join_date_clean', axis = 1)

# Running the imputer
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors = 5)
df2_imputed = imputer.fit_transform(df2)

df2_imputed = pd.DataFrame(df2_imputed, columns = df2.columns)


In [12]:
# Confirming blanks have been removed
df2_imputed.isna().sum()

country                         0
age                             0
gender                          0
subscription_type               0
avg_listening_hours_per_week    0
total_songs_played              0
skip_rate                       0
satisfaction_score              0
churned                         0
new_avg_price                   0
dtype: int64

In [13]:
df2_imputed.head()

,country,age,gender,subscription_type,avg_listening_hours_per_week,total_songs_played,skip_rate,satisfaction_score,churned,new_avg_price
0,4.0,-1.016941,0.0,2.0,0.306573,0.486594,-1.486964,1.0,0.0,2.0
1,4.0,1.609359,1.0,3.0,0.264996,1.198434,-1.486964,2.0,0.0,0.0
2,8.0,0.427524,0.0,2.0,-1.668348,-1.026067,1.663383,2.0,0.0,2.0
3,5.0,-1.476543,0.0,0.0,0.431305,1.331904,0.403244,4.0,1.0,4.0
4,1.0,-0.622996,1.0,1.0,-0.753648,-0.981577,0.403244,2.0,0.0,0.0


# Clustering


In [14]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters = 4, random_state = 7)
df2_imputed['cluster'] = kmeans.fit_predict(df2_imputed)

# Undoing the ordinal encoding to get back actual categories
df_original_cats = pd.DataFrame(
    enc.inverse_transform(df2_imputed[cat_cols]),
    columns=cat_cols
)

# Undoing the scaling to get back actual numerical values
df_original_nums = pd.DataFrame(
    scaler.inverse_transform(df2_imputed[num_cols]),
    columns=num_cols
)

# combining cluster, date and user_id fields
df_readable = pd.concat([df_original_nums, df_original_cats, df2_imputed['cluster']], axis=1)
df_readable['Join_date_clean'] = dates
df_readable['user_id'] = user_id

C:\Users\mites\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [15]:
df_readable.head()

,age,avg_listening_hours_per_week,total_songs_played,skip_rate,country,gender,subscription_type,satisfaction_score,churned,new_avg_price,cluster,Join_date_clean,user_id
0,23.0,11.5,510.0,0.05,Nepal,Female,Premium,2.0,0,7.99,0,2024-03-21,U1403
1,63.0,11.3,526.0,0.05,Nepal,Male,Student,3.0,0,0.0,2,2023-08-10,U1165
2,45.0,2.0,476.0,0.30,USA,Female,Premium,3.0,0,7.99,1,2022-04-05,U1474
3,16.0,12.1,529.0,0.20,Nigeria,Female,Family,5.0,1,12.99,1,2023-12-16,U1478
4,29.0,6.4,477.0,0.20,Ghana,Male,Free,3.0,0,0.0,2,2022-03-28,U1609


In [17]:
# Exporting data file so we can analyse in PowerBI
df_readable.to_excel('cleaned_final.xlsx', index= False)